# Fusionar los datos scrapeados

Fusionar los datos recolectados para descargar los videos.

In [1]:
import sys
import os

# Agregar el directorio raiz al PYTHONPATH
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import os
import pandas as pd
from tqdm import tqdm

from constants.constants_tiktok import TIKTOK_SCRAPED_DIR
EXT = ".json"

dfs = []
# Cargar archivos y combinarlos
for filename in tqdm(os.listdir(TIKTOK_SCRAPED_DIR)):
    if filename.endswith(EXT):
        full_path = os.path.join(TIKTOK_SCRAPED_DIR, filename)
        df = pd.read_json(full_path, encoding="utf-8")
        termino = filename.replace(EXT, "")
        df['termino'] = "_".join(termino.lower().split())
        dfs.append(df)
merged_df = pd.concat(dfs, ignore_index=True)
print(merged_df.info())

100%|██████████| 28/28 [00:00<00:00, 145.62it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7108 entries, 0 to 7107
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   views               7108 non-null   int64 
 1   title               7108 non-null   object
 2   hashtags            7108 non-null   object
 3   date                7108 non-null   object
 4   fecha_recuperacion  7108 non-null   object
 5   caption             7108 non-null   object
 6   url                 7108 non-null   object
 7   termino             7108 non-null   object
dtypes: int64(1), object(7)
memory usage: 444.4+ KB
None


Eliminar filas donde las urls estén repetidas para evitar descargar un video varias veces

In [3]:
merged_df.drop_duplicates(subset="url", keep="first").reset_index(drop=True)
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7108 entries, 0 to 7107
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   views               7108 non-null   int64 
 1   title               7108 non-null   object
 2   hashtags            7108 non-null   object
 3   date                7108 non-null   object
 4   fecha_recuperacion  7108 non-null   object
 5   caption             7108 non-null   object
 6   url                 7108 non-null   object
 7   termino             7108 non-null   object
dtypes: int64(1), object(7)
memory usage: 444.4+ KB
None



Guardar archivos recolectados combinados

In [5]:

from constants.constants_tiktok import TIKTOK_MERGED_SCRAPED_PATH

merged_df.to_csv(TIKTOK_MERGED_SCRAPED_PATH, index=False)
print(f"Archivo guardado en: {TIKTOK_MERGED_SCRAPED_PATH}")

Archivo guardado en: c:\Users\Diego\Desktop\sentiment_analysis\data\tiktok\raw\tiktok_merged_scraped.csv


# Descargar y transcribir videos

In [ ]:
from scraping.data_collection import download_and_trancribe

# Cargar datos a descargar
merged_df = pd.read_csv(TIKTOK_MERGED_SCRAPED_PATH, encoding="utf-8")

start = 0
end = 1

merged_df['descargado'] = None
merged_df['text'] = None
merged_df['idioma'] = None

sub_df = merged_df[start:end]
sub_df.apply(download_and_trancribe, axis=1)

